In [4]:
#평가 스케일
def rmsle(y, y_,convertExp=True):
    if convertExp:
        y = np.exp(y), 
        y_ = np.exp(y_)
    log1 = np.nan_to_num(np.array([np.log(v + 1) for v in y]))
    log2 = np.nan_to_num(np.array([np.log(v + 1) for v in y_]))
    calc = (log1 - log2) ** 2
    return np.sqrt(np.mean(calc))

from sklearn import metrics
rmsle_scorer = metrics.make_scorer(rmsle)

# 0 전처리

In [5]:
import pandas as pd
import numpy as np

In [6]:
train = pd.read_csv("/Users/yoonjeong/Desktop/af_train_0.csv", encoding='cp949')
test = pd.read_csv("/Users/yoonjeong/Desktop/af_test_0.csv", encoding='cp949')

모델링

In [7]:
# X : features, y: target variable
X_train=  train[train.columns.difference(["이용자 수"])]
X_test = test[test.columns.difference(["이용자 수"])]
y_train= train["이용자 수"]
y_test= test['이용자 수']

# 0 random forest

In [8]:
from sklearn.model_selection import RandomizedSearchCV

n_estimators = [int(x) for x in np.linspace(start = 10, stop = 30, num = 5)]

max_features = ['auto', 'sqrt']

max_depth = [int(x) for x in np.linspace(10, 50, num = 5)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [10, 15, 20, 25, 30], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, None], 'min_samples_split': [2, 5], 'min_samples_leaf': [1, 2], 'bootstrap': [True, False]}


In [9]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
grid_rf = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 10, verbose=2, random_state=42, n_jobs = -1,  scoring=rmsle_scorer)
grid_rf.fit(X_train, y_train)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


/Users/yoonjeong/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf]
  warnings.warn(
/Users/yoonjeong/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:925: RuntimeWarning: invalid value encountered in subtract
  array_stds = np.sqrt(np.average((array -


RandomizedSearchCV(cv=10, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2],
                                        'min_samples_split': [2, 5],
                                        'n_estimators': [10, 15, 20, 25, 30]},
                   random_state=42, scoring=make_scorer(rmsle), verbose=2)

In [10]:
pred = grid_rf.predict(X_train)
print('RMSLE Value For Random Forest_af_group0: %.4f' % (rmsle(pred,y_train,False)))
print("\n")


y_pred = grid_rf.predict(X_test)
print('RMSLE Value For Random Forest_af_group0 : %.4f' % (rmsle(y_pred,y_test,False)))
print("\n")

RMSLE Value For Random Forest_af_group0: 0.1277


RMSLE Value For Random Forest_af_group0 : 0.2463




In [11]:
import pandas as pd
rf1 = grid_rf.best_estimator_.fit(X_train, y_train)
feature_imp = pd.DataFrame(rf1.feature_importances_,index=X_train.columns, columns=['importance']).sort_values('importance',ascending=False)
feature_imp

,importance
휴일,0.183264
반석,0.112681
술집,0.103692
올리브영,0.100386
출구,0.067544
샌드위치,0.064935
버스정류장,0.054718
날짜,0.048061
확진자수,0.043293
은행,0.036192


# 1 전처리

In [12]:
import pandas as pd
import numpy as np

In [13]:
train = pd.read_csv("/Users/yoonjeong/Desktop/af_train_1.csv", encoding='cp949')
test = pd.read_csv("/Users/yoonjeong/Desktop/af_test_1.csv", encoding='cp949')

모델링

In [14]:
# X : features, y: target variable
X_train=  train[train.columns.difference(["이용자 수"])]
X_test = test[test.columns.difference(["이용자 수"])]
y_train= train["이용자 수"]
y_test= test['이용자 수']

# 1 random forest

In [15]:
from sklearn.model_selection import RandomizedSearchCV

n_estimators = [int(x) for x in np.linspace(start = 10, stop = 30, num = 5)]

max_features = ['auto', 'sqrt']

max_depth = [int(x) for x in np.linspace(10, 50, num = 5)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [10, 15, 20, 25, 30], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, None], 'min_samples_split': [2, 5], 'min_samples_leaf': [1, 2], 'bootstrap': [True, False]}


In [16]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
grid_rf = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 10, verbose=2, random_state=42, n_jobs = -1,  scoring=rmsle_scorer)
grid_rf.fit(X_train, y_train)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


RandomizedSearchCV(cv=10, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2],
                                        'min_samples_split': [2, 5],
                                        'n_estimators': [10, 15, 20, 25, 30]},
                   random_state=42, scoring=make_scorer(rmsle), verbose=2)

In [17]:
pred = grid_rf.predict(X_train)
print('RMSLE Value For Random Forest_af_group1: %.4f' % (rmsle(pred,y_train,False)))
print("\n")


y_pred = grid_rf.predict(X_test)
print('RMSLE Value For Random Forest_af_group1 : %.4f' % (rmsle(y_pred,y_test,False)))
print("\n")

RMSLE Value For Random Forest_af_group1: 0.1358


RMSLE Value For Random Forest_af_group1 : 0.2649




In [18]:
import pandas as pd
rf1 = grid_rf.best_estimator_.fit(X_train, y_train)
feature_imp = pd.DataFrame(rf1.feature_importances_,index=X_train.columns, columns=['importance']).sort_values('importance',ascending=False)
feature_imp

,importance
휴일,0.177849
은행,0.113629
영화관_b,0.100176
출구,0.064726
올리브영,0.063164
학교,0.061302
현충원,0.045083
노령화지수,0.044855
확진자수,0.043818
날짜,0.043213


# 2 전처리

In [19]:
import pandas as pd
import numpy as np

In [20]:
train = pd.read_csv("/Users/yoonjeong/Desktop/af_train_2.csv", encoding='cp949')
test = pd.read_csv("/Users/yoonjeong/Desktop/af_test_2.csv", encoding='cp949')

모델링

In [21]:
# X : features, y: target variable
X_train=  train[train.columns.difference(["이용자 수"])]
X_test = test[test.columns.difference(["이용자 수"])]
y_train= train["이용자 수"]
y_test= test['이용자 수']

# 2 random forest

In [22]:
from sklearn.model_selection import RandomizedSearchCV

n_estimators = [int(x) for x in np.linspace(start = 10, stop = 30, num = 5)]

max_features = ['auto', 'sqrt']

max_depth = [int(x) for x in np.linspace(10, 50, num = 5)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [10, 15, 20, 25, 30], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, None], 'min_samples_split': [2, 5], 'min_samples_leaf': [1, 2], 'bootstrap': [True, False]}


In [23]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
grid_rf = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 10, verbose=2, random_state=42, n_jobs = -1,  scoring=rmsle_scorer)
grid_rf.fit(X_train, y_train)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


/Users/yoonjeong/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf]
  warnings.warn(
/Users/yoonjeong/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:925: RuntimeWarning: invalid value encountered in subtract
  array_stds = np.sqrt(np.average((array -


RandomizedSearchCV(cv=10, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2],
                                        'min_samples_split': [2, 5],
                                        'n_estimators': [10, 15, 20, 25, 30]},
                   random_state=42, scoring=make_scorer(rmsle), verbose=2)

In [24]:
pred = grid_rf.predict(X_train)
print('RMSLE Value For Random Forest_af_group2: %.4f' % (rmsle(pred,y_train,False)))
print("\n")


y_pred = grid_rf.predict(X_test)
print('RMSLE Value For Random Forest_af_group2 : %.4f' % (rmsle(y_pred,y_test,False)))
print("\n")

RMSLE Value For Random Forest_af_group2: 0.1007


RMSLE Value For Random Forest_af_group2 : 0.2285




In [25]:
import pandas as pd
rf1 = grid_rf.best_estimator_.fit(X_train, y_train)
feature_imp = pd.DataFrame(rf1.feature_importances_,index=X_train.columns, columns=['importance']).sort_values('importance',ascending=False)
feature_imp

,importance
갑천,0.141776
버스정류장,0.132417
휴일,0.118796
은행,0.110118
출구,0.101838
술집,0.084428
날짜,0.052669
확진자수,0.051448
인구수,0.035565
최저기온,0.026952


# 3 전처리

In [26]:
import pandas as pd
import numpy as np

In [27]:
train = pd.read_csv("/Users/yoonjeong/Desktop/af_train_3.csv", encoding='cp949')
test = pd.read_csv("/Users/yoonjeong/Desktop/af_test_3.csv", encoding='cp949')

모델링

In [28]:
# X : features, y: target variable
X_train=  train[train.columns.difference(["이용자 수"])]
X_test = test[test.columns.difference(["이용자 수"])]
y_train= train["이용자 수"]
y_test= test['이용자 수']

# 3 random forest

In [29]:
from sklearn.model_selection import RandomizedSearchCV

n_estimators = [int(x) for x in np.linspace(start = 10, stop = 30, num = 5)]

max_features = ['auto', 'sqrt']

max_depth = [int(x) for x in np.linspace(10, 50, num = 5)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [10, 15, 20, 25, 30], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, None], 'min_samples_split': [2, 5], 'min_samples_leaf': [1, 2], 'bootstrap': [True, False]}


In [30]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
grid_rf = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 10, verbose=2, random_state=42, n_jobs = -1,  scoring=rmsle_scorer)
grid_rf.fit(X_train, y_train)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


/Users/yoonjeong/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf]
  warnings.warn(
/Users/yoonjeong/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:925: RuntimeWarning: invalid value encountered in subtract
  array_stds = np.sqrt(np.average((array -


RandomizedSearchCV(cv=10, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2],
                                        'min_samples_split': [2, 5],
                                        'n_estimators': [10, 15, 20, 25, 30]},
                   random_state=42, scoring=make_scorer(rmsle), verbose=2)

In [31]:
pred = grid_rf.predict(X_train)
print('RMSLE Value For Random Forest_af_group3: %.4f' % (rmsle(pred,y_train,False)))
print("\n")


y_pred = grid_rf.predict(X_test)
print('RMSLE Value For Random Forest_af_group3 : %.4f' % (rmsle(y_pred,y_test,False)))
print("\n")

RMSLE Value For Random Forest_af_group3: 0.1198


RMSLE Value For Random Forest_af_group3 : 0.2389




In [32]:
import pandas as pd
rf1 = grid_rf.best_estimator_.fit(X_train, y_train)
feature_imp = pd.DataFrame(rf1.feature_importances_,index=X_train.columns, columns=['importance']).sort_values('importance',ascending=False)
feature_imp

,importance
노령화지수,0.128179
휴일,0.127411
샌드위치,0.095033
버스정류장,0.093321
은행,0.080670
인구수,0.067240
확진자수,0.057676
올리브영,0.046753
날짜,0.046585
영화관_b,0.036068
